In [2]:
from dotenv import load_dotenv
from conversationService import get_embedding
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import loggingService
import os
import weaviate

load_dotenv()
logger = loggingService.get_logger()


/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
rufimelo/bert-large-portuguese-cased-sts


/Users/vanildovanni/miniconda3/envs/ml/lib/python3.9/site-packages/genai/credentials.py:31: DeprecationWarning: The 'api_endpoint' property should not contain any explicit API version(rename it from 'https://workbench-api.res.ibm.com/v1' to just 'https://workbench-api.res.ibm.com')
  warn(


2023-09-29 11:16:56,319 - numexpr.utils - INFO - Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-09-29 11:16:56,319 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [3]:
apikey = os.getenv("GEANAI_KEY", None)
class_name = os.getenv("WEVIATE_CLASS", 'LivrosVectorizer')
path = os.getenv("DATA_PATH", "data")
weaviate_url = os.getenv("WEAVIATE_URL", 'http://127.0.0.1:8080')

client = weaviate.Client(
    url=weaviate_url,
)

pages = []
pdf_loader = PyPDFDirectoryLoader(path)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50, separators=["\n\n", "\n", " ", "", "\n \n", "\n \n \n","\n\n\n"])
documents = pdf_loader.load_and_split(text_splitter=text_splitter)
print(len(documents))
print(documents[0])

1137
page_content='Dom Casmu rro\nMachado de  Assi s\nCAPÍTULO PRIM EIRO / DO TÍTULO \nUma noite destas, vindo da  cidade para o Engenho N ovo, e ncontrei num  trem da Central um rapaz \naqui do ba irro, que  eu conhe ço de vista e de chapéu. Cum primentou-m e, sentou-se ao pé de mim, \nfalou da  lua e dos ministros, e acabou re citando-m e versos. A viagem era curta, e os versos pode  ser' metadata={'source': 'data/bv00180a.pdf', 'page': 0}


In [ ]:
def pdf_text_splitter(pdf_text) -> dict:
  retorno = {'content': '', 'source': '', 'page': 0}
  
  retorno['content'] = getattr(pdf_text, 'page_content')
  retorno['souce'] = getattr(pdf_text, 'metadata')['source']
  retorno['page'] = getattr(pdf_text, 'metadata')['page']
  
  return retorno

for doc in documents:
  # logger.debug(pdf_text_splitter(doc))
  # print(pdf_text_splitter(doc))
  pages.append(pdf_text_splitter(doc))
  
print(pages[0])

In [3]:
class_document =  {
  "class": "Livros",
  "description": "Dados do documento PDF",
  "moduleConfig": {
    "text2vec-transformers": {
      "poolingStrategy": "masked_mean",
      "vectorizeClassName": False
    }
  },
  "properties": [
    {
      "dataType": [
        "text"
      ],
      "description": "Conteúdo do documento PDF",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "content"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "source"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "page"
    }
  ],
  "vectorizer": "none"
}

class_document2 =  {
  "class": "LivrosVectorizer",
  "description": "Dados do documento PDF",
  "moduleConfig": {
    "text2vec-transformers": {
      "poolingStrategy": "masked_mean",
      "vectorizeClassName": False
    }
  },
  "properties": [
    {
      "dataType": [
        "text"
      ],
      "description": "Conteúdo do documento PDF",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "content"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "source"
    },
    {
      "dataType": [
        "text"
      ],
      "description": "",
      "moduleConfig": {
        "text2vec-transformers": {
          "skip": False,
          "vectorizePropertyName": False
        }
      },
      "name": "page"
    }
  ],
  "vectorizer": "text2vec-transformers"
}

client.schema.create_class(class_document)
# client.schema.delete_class('Livros')
# client.schema.get("LivrosVectorizer")
# client.schema.property.create(class_name, {
#     'name': 'exemplar',
#     'dataType': ['text'],
# })


In [ ]:
client.batch.configure(batch_size=10)  # Configure batch
with client.batch as batch:
  i = 0
    
  for page in pages:
    logger.info(f"importing question: {i+1}")
    i = i+1
    
    properties = {
      "content": page["content"],
      "page": str(page["page"]),
      "source": page["source"],
    }
    
    vector = get_embedding(page["content"])

    client.batch.add_data_object(properties, 'Livros', vector=vector)
    # client.batch.add_data_object(properties, 'LivrosVectorizer')

In [ ]:
result = (client.query
.get('Livros', ["content", "source", "page"])
.with_additional(["certainty", "distance"]) # note that certainty is only supported if distance==cosine
.with_near_vector({
  "vector": get_embedding('o que significa casmurro'),
  "certainty": 0.8
})
.with_limit(4)
.do()
)

print(result['data']['Get'][class_name])

retorno = ''
if len(result['data']['Get'][class_name]) > 0:
  retorno = result['data']['Get'][class_name][0]['content']

  for contexto in result['data']['Get'][class_name][1:]:
    retorno += f"\n{contexto['content']}"

logger.info(retorno)


In [23]:
print(client.query.aggregate('Livros').with_meta_count().do())

result = (client.query
  .get('Livros', ["content", "source", "page"])
  .with_additional(["certainty", "distance"]) # note that certainty is only supported if distance==cosine
  .with_near_vector({
    "vector": get_embedding('um vasto círculo de saias ou, t repadas no a r, choviam pés e pernas sobre a minha cabeça. Assim fui \naté madruga da. Não dorm i mais;'),
    "certainty": 0.8
  })
  .with_limit(4)
  .do()
)

print(result)
print(result['data']['Get']['Livros'])

{'data': {'Aggregate': {'Livros': [{'meta': {'count': 1119}}]}}}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'data': {'Get': {'Livros': [{'_additional': {'certainty': 0.8990992605686188, 'distance': 0.20180148}, 'content': 'um vasto círculo de saias ou, t repadas no a r, choviam pés e pernas sobre a minha cabeça. Assim fui \naté madruga da. Não dorm i mais; rezei padre-nossos, ave-marias, e credos, e sendo e ste livro a \nverdade pura , é força confessar que  tive de interrom per mais de uma vez as minhas orações para \nacompanhar no e scuro um a figura ao longe , tique-tique, tique-tique... Pegava depressa na oração,', 'page': '46', 'source': ''}, {'_additional': {'certainty': 0.8355421423912048, 'distance': 0.32891572}, 'content': 'carro andaria mais depressa que as pernas- estas iriam pausadas ou nã o, podi a afrouxa r o pa sso. \nparar, arrepiar caminho, e  deixar que  a cabeça cismasse à vont ade. Fui andando e  cismando. T inha \njá comparado o ge sto de Sancha na véspera e o de sespero da quele dia; eram inconciliáveis. A viúva', 'page': '91', 'source': ''}, {'_additional': {'certaint

In [24]:
result = client.data_object.get(class_name='Livros', offset=300, with_vector=True)

print(result['objects'][0]['properties'])
print(len(result['objects'][0]['vector']))

{'content': '--Com o? \n--Espere até amanhã. Vou jogar com eles que me chamaram, amanhã, lá no qua rto, no qui ntal, ou na \nrua, indo à  missa, conto-lhe o que  há. A idéia é tão santa que não está mal no s antuário. Amanha, \nBentinho. \n-- Mas é cousa certa? \n--Certíssima! \nNo dia seguinte revelou-m e o mistério. Ao primeiro aspecto confesso que  fiquei deslumbrado.', 'page': '71', 'source': ''}
1024
